# Harami Cross Candle

In [7]:
import pandas as pd
import numpy as np
import talib
import plotly.graph_objects as go
import datetime

In [8]:
df = pd.read_csv("nifty5min.csv")
#df = df[(df.Date > '2022-09-04 9:00:00') & (df.Date < '2022-09-05 15:30:00')]
df.set_index("Date", inplace=True)
df = df[['Open', 'High', 'Low', 'Close']]
df.head(10)

,Open,High,Low,Close
Date,,,,
2022-07-19 09:15:00,16195,16243,16195,16240
2022-07-19 09:20:00,16240,16253,16233,16252
2022-07-19 09:25:00,16252,16259,16245,16258
2022-07-19 09:30:00,16258,16267,16251,16253
2022-07-19 09:35:00,16254,16254,16240,16242
2022-07-19 09:40:00,16242,16268,16241,16265
2022-07-19 09:45:00,16265,16279,16264,16270
2022-07-19 09:50:00,16271,16289,16270,16285
2022-07-19 09:55:00,16284,16284,16266,16277


In [10]:
harami = talib.CDLHARAMICROSS(df['Open'], df['High'], df['Low'], df['Close'])
df['harami']= harami
df['SignalGreen'] = False
df['SignalRed'] = False
df['body'] = abs(df['Open'] - df['Close'])

def createsignal(harami):
    import numpy as np
    SignalGreen   = []
    SignalRed = []
    i = 0
    for date,value in harami.iteritems():
        if value == 100:
            SignalGreen.append(df['High'].iloc[i] + 10)
            df['SignalGreen'].iat[i] = True
            SignalRed.append(np.nan)
        elif value == -100:
            SignalRed.append(df['High'].iloc[i] + 10)
            SignalGreen.append(np.nan)
            df['SignalRed'].iat[i] = True
        else:
            SignalGreen.append(np.nan)
            SignalRed.append(np.nan)
        i += 1
    return SignalGreen, SignalRed

In [11]:
harami[harami!=0]

Date
2022-07-21 13:30:00    100
2022-07-21 14:05:00   -100
2022-07-21 15:30:00    100
2022-07-22 10:40:00    100
2022-07-22 15:30:00   -100
2022-07-27 11:10:00    100
2022-07-28 12:50:00   -100
2022-08-01 11:40:00   -100
2022-08-01 14:30:00   -100
2022-08-02 12:55:00   -100
2022-08-03 13:45:00    100
2022-08-05 10:55:00   -100
2022-08-05 15:30:00    100
2022-08-08 13:15:00    100
2022-08-10 11:30:00   -100
2022-08-11 13:50:00   -100
2022-08-11 15:30:00    100
2022-08-16 09:40:00   -100
2022-08-16 14:35:00    100
2022-08-17 14:50:00   -100
2022-08-19 15:30:00    100
2022-08-23 12:55:00   -100
2022-08-26 09:55:00   -100
2022-08-26 13:10:00    100
2022-08-26 15:20:00   -100
2022-08-29 15:30:00   -100
2022-08-30 15:30:00    100
2022-09-02 14:10:00   -100
2022-09-02 15:20:00    100
2022-09-08 13:00:00    100
2022-09-09 09:25:00    100
2022-09-09 15:05:00    100
dtype: int32

In [12]:
SignalGreen, SignalRed = createsignal(harami)
x = df.index.tolist()

In [13]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df.Open, high=df.High,
                low=df.Low, close=df.Close)
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    width=800, height=600,
    title="Nifty",
    yaxis_title='Price',
    xaxis=dict(type = "category")
)

fig.add_trace(go.Scatter(
    x=x,
    y=SignalGreen,
    name ='harami Green',
    mode="markers", 
    marker_size=10,
    marker = dict(color='green', symbol = "square-dot")   
))

fig.add_trace(go.Scatter(
    x=x,
    y=SignalRed,
    name ='harami Red',
    mode="markers", 
    marker_size=10,
    marker = dict(color='red', symbol = "square-dot")   
))

fig.show()
